In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-129211")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-129211
Azure region: southcentralus
Subscription id: c1dcf523-8ab0-429f-8dba-143a6c49ec84
Resource group: aml-quickstarts-129211


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Reference: lesson 5.9 and Microsoft website at:
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute(class)?view=azure-ml-py

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                          max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [30]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import normal,uniform,choice
import os
import shutil

# Specify parameter sampler
ps = RandomParameterSampling( 
     {
    '--C': choice(0.0001,0.001,0.01,0.1,1,10,100,1000),
    '--max_iter': choice(100,200,300,400,500)
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1,delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=".",compute_target=cpu_cluster,entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=100,
                             max_concurrent_runs=4)

In [31]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config,show_output=True)
RunDetails(hyperdrive_run).show()


WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [44]:
import joblib


# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()

best_run_metrics = best_run.get_metrics()
params = best_run.get_details()['runDefinition']['arguments']
print('Id of the best run: ', best_run.id)
print('Accuracy of the best run:', best_run_metrics['Accuracy'])
best_run.download_file("model.joblib", "linreg_best_model.joblib")

Id of the best run:  HD_0d8f579d-fe25-451b-901a-2fe7d9d430ed_1
Accuracy of the best run: 0.9121396054628225


UserErrorException: UserErrorException:
	Message: File with path model.joblib was not found,
available files include: azureml-logs/55_azureml-execution-tvmps_f916888f1694a6597232e7fb857c599cb9925a6407049a2e11f6215bf79a07c7_d.txt,azureml-logs/65_job_prep-tvmps_f916888f1694a6597232e7fb857c599cb9925a6407049a2e11f6215bf79a07c7_d.txt,azureml-logs/70_driver_log.txt,azureml-logs/75_job_post-tvmps_f916888f1694a6597232e7fb857c599cb9925a6407049a2e11f6215bf79a07c7_d.txt,azureml-logs/process_info.json,azureml-logs/process_status.json,logs/azureml/101_azureml.log,logs/azureml/job_prep_azureml.log,logs/azureml/job_release_azureml.log.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "File with path model.joblib was not found,\navailable files include: azureml-logs/55_azureml-execution-tvmps_f916888f1694a6597232e7fb857c599cb9925a6407049a2e11f6215bf79a07c7_d.txt,azureml-logs/65_job_prep-tvmps_f916888f1694a6597232e7fb857c599cb9925a6407049a2e11f6215bf

In [35]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

datastore_path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path=datastore_path)




In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

from sklearn.model_selection import train_test_split

import pandas as pd

x_train, x_test, y_train, y_test = train_test_split(x, y)

training_data = pd.concat([x_train, y_train], axis=1)

In [ ]:
training_data.head()

In [ ]:
training_data.describe()

In [ ]:
training_data[training_data.isnull().any(axis=1)]

In [ ]:
training_data["y"].value_counts()

In [ ]:
from azureml.core import Workspace, Dataset

#creating a folder
#dirname = "./training_data"
os.makedirs('training_data', exist_ok=True)

local_path = './training_data/training_data.csv'
training_data.to_csv(local_path)

# upload the local file to a datastore on the cloud
workspace = Workspace(ws.subscription_id, ws.resource_group, ws.name)

# get the datastore to upload prepared data
datastore = ws.get_default_datastore()

# upload the local file from src_dir to the target_path in datastore
datastore.upload(src_dir='training_data', target_path='training_data')

# create a dataset referencing the cloud location

ds = TabularDatasetFactory.from_delimited_files(path=[(datastore, ('training_data/training_data.csv'))])

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric="accuracy",
    training_data=training_data,
    label_column_name="y",
    n_cross_validations=5)

In [ ]:
# Submit your automl run
exp = Experiment(workspace=ws, name="AutoML-project")
automl_run=exp.submit(automl_config,show_output=True)
RunDetails(automl_run).show()
assert(automl_run.get_status() == "Completed")

In [ ]:
# Retrieve and save your best automl model.

from azureml.automl.runtime.onnx_convert import OnnxConverter
best_aml_marketing_model, onnx_model= automl_run.get_output(return_onnx_model=True)
OnnxConverter.save_onnx_model(onnx_model, "./automl_model.onnx")

In [ ]:
cpu_cluster.delete()